In [3]:
%pip install requests flet

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached flet-0.28.3-py3-none-any.whl.metadata (3.6 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-macosx_10_13_universal2.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached oauthlib-3.3.1-py3-none-any.whl.metadata (7.9 kB)
  Using cached repath-0.9.0-py3-none-any.whl.metadata (899 bytes)
  Using cached anyio-4.12.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached flet-0.28.3-py3-none-any.whl (463 kB)
Using cached certifi-2025.11.12-py3-none-any.whl (159 kB)
Using cached charset_norma

In [4]:
import requests
import flet as ft

def main(page: ft.Page):
    page.title = "気象庁天気予報アプリ"
    
    # --- 天気情報を表示するための領域 ---
    result_text = ft.Text("地域を選択してください", size=20, weight="bold")
    weather_detail = ft.Column()

    def get_weather(code, name):
        try:
            url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{code}.json"
            data = requests.get(url).json()
            # 取得したデータから最初の予報を抽出
            weather_forecast = data[0]["timeSeries"][0]["areas"][0]["weathers"][0]
            
            result_text.value = f"【{name}】の天気"
            weather_detail.controls.clear()
            weather_detail.controls.append(ft.Text(weather_forecast, size=16))
            page.update()
        except Exception as ex:
            result_text.value = f"取得エラー: {ex}"
            page.update()

    # --- 地域データの取得と加工 ---
    area_url = "https://www.jma.go.jp/bosai/common/const/area.json"
    area_data = requests.get(area_url).json()
    centers = area_data["centers"]  # 地方区分（関東甲信など）
    offices = area_data["offices"]  # 都道府県区分

    def on_area_select(e):
        # 選択された地域のコードと名前を取得
        area_code = e.control.data["code"]
        area_name = e.control.data["name"]
        get_weather(area_code, area_name)

    # --- UIの構築 (ExpansionTileを使用) ---
    lv = ft.ListView(expand=1, spacing=5)
    
    for c_code, c_info in centers.items():
        # 地方ごとのExpansionTileを作成
        exp = ft.ExpansionTile(
            title=ft.Text(c_info["name"]),
            subtitle=ft.Text(f"{len(c_info['children'])} 地域"),
            maintain_state=True,
            controls=[]
        )
        
        # 地方に含まれる県(offices)を追加
        for o_code in c_info["children"]:
            if o_code in offices:
                exp.controls.append(
                    ft.ListTile(
                        title=ft.Text(offices[o_code]["name"]),
                        data={"code": o_code, "name": offices[o_code]["name"]},
                        on_click=on_area_select,
                    )
                )
        lv.controls.append(exp)

    # レイアウト配置
    page.add(
        ft.Row(
            [
                ft.Container(content=lv, width=300, border=ft.border.all(1, "grey100")),
                ft.VerticalDivider(width=1),
                ft.Container(
                    content=ft.Column([result_text, weather_detail], scroll=ft.ScrollMode.AUTO),
                    expand=True,
                    padding=20
                )
            ],
            expand=True
        )
    )

# Notebook環境での実行（ブラウザで開く設定）
ft.app(target=main, view=ft.AppView.WEB_BROWSER)

RuntimeError: asyncio.run() cannot be called from a running event loop